In [1]:
import wget
import os
from selenium import webdriver
from datetime import datetime

from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [43]:
PATH = r'C:\Program Files (x86)\chromedriver.exe'
#driver = webdriver.Chrome(PATH)
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(PATH, options=chrome_options)

In [8]:
### Warning!!! Will erase all data!!!
import pandas as pd
data = [{'Suburb': 0, 'Address': 0, 'Description': 0, 'Price': 0, 'Amenities': 0, 'Type': 0, 'Date': 0, 'Insights': 0, 'Schools': 0, 'URL': 0, 'Profile': 0, "Performance": 0}] 
listings = pd.DataFrame(data)
listings.head()

,Suburb,Address,Description,Price,Amenities,Type,Date,Insights,Schools,URL,Profile,Performance
0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
timestamp = 1545730073
dt_object = datetime.fromtimestamp(timestamp)

In [131]:
### Scrapper in Full

### Debug
add_record_count = 0
time = datetime.now
file = open("ErrorLog.txt", "a")
file.write('\nRan at ')
file.write(str(dt_object))
file2 = open("UrlLog.txt", "a")

#Landing page for melbourne house prices
page_link = "https://www.domain.com.au/auction-results/melbourne/"

for num_1 in range(3,4):
    for num_2 in range(1,2):
        #looping through suburbs
        
        #Going to melbourne landing page
        driver.get(page_link)
        
        #Finding and clicking on the suburb listing
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/main/section/div/div[2]/div[2]/div/div/div/div[' + str(num_1) + ']/article[' + str(num_2) + ']/div/a[1]')))
            driver.find_element_by_xpath('/html/body/div[1]/div/main/section/div/div[2]/div[2]/div/div/div/div[' + str(num_1) + ']/article[' + str(num_2) + ']/div/a[1]').click()
            
        except:
            break
        
        #Grabbing the name of the suburb
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[1]/h1')))
            element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[1]/h1')
            suburb = element.text
        except:
            pass
        
        #Going through pages of listings within suburbs
        try:
            current_url = driver.current_url 
            for page in range(1,10):
                #looping through pages of listings with a suburb
                if page == 1:
                    driver.get(current_url)
                else:
                    try:
                        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[1]/div/div[1]/div[1]/div/div/div/div/div[2]/div/a/div/img')))
                        driver.get(current_url + "?page=" + str(page))
                    except:
                        file.write("\nGoing through pages of listings within suburbs broken at page" + str(page))
                        break
                
                reset_point = driver.current_url
                

                for listing_num in range(1,25):
                #looping through listings in a page
                    driver.get(reset_point)
                    try:
                        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[' + str(listing_num) + ']/div/div[2]/div/a/h2/span[1]')))
                        driver.find_element_by_xpath('/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[' + str(listing_num) + ']/div/div[2]/div/a/h2/span[1]').click()

                        #Grabs listing description
                        try:
                            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//button[@data-testid="listing-details__description-button"]')))
                            driver.find_element_by_xpath('//button[@data-testid="listing-details__description-button"]').send_keys(u'\ue007')
                            WebDriverWait(driver, 1)
                        except:
                            file.write('\nFailed to expand description in url' + str(driver.current_url) + "Page" + str(page) + "listing" + str(listing_num))
                            pass
                        listing_description = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[5]/div/div/div[3]/div[1]/div/div/div').text

                        #Grabs listing's address
                        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[2]/div/div[1]')
                        listing_address = element.text

                        #Grabs listing's price
                        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[1]/div[1]/div')
                        listing_price = element.text

                        #Grabs listing's Amenities
                        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div')
                        listing_amenities = element.text

                        #Grabs listing's property type
                        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[4]/span')
                        listing_property_type = element.text

                        #Grabs listing's sold date
                        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/span[2]')
                        listing_sold_date = element.text

                        try:
                            #Grabs listing's domain insight
                            element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[5]/div/div/p')
                            listing_domain_insight = element.text

                            #Grabs School details
                            driver.find_element_by_xpath('//button[@data-testid="fe-co-school-catchment-view-more-link"]').send_keys(u'\ue007')
                            element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[9]/div/div/section/div[3]/ul')
                            listing_schools = element.text
                        except:
                            file.write('\nFailed to expand schools in url' + str(driver.current_url) + "Page" + str(page) + "listing" + str(listing_num))
                            pass

                        #Grabs listing url
                        listing_url = driver.current_url
                        file2.write("\n" + listing_url)

                        #Storing data
                        data = [{'Suburb': suburb,'Address': listing_address, 'Description': listing_description, 'Price': listing_price, 'Amenities': listing_amenities, 'Type': listing_property_type, 'Date': listing_sold_date, 'Insights': listing_domain_insight, 'Schools': listing_schools, 'URL': listing_url}] 
                        output = pd.DataFrame(data)    
                        listings = listings.append(output, ignore_index=True)

                        add_record_count += 1
                        print(add_record_count)


                        listings.to_csv("databackup.csv", encoding='utf-8', index=False)

                    except:
                        file.write("\nError - " + str(listing_num) + " Not found in page" + str(page))
                        pass

        except:
            file.write('\nUncaptured Error')
            raise
file.close()
file2.close()

NameError: name 'dt_object' is not defined

In [ ]:
#Landing page for melbourne house prices
page_link = "https://www.domain.com.au/sold-listings/abbotsford-vic-3067/" 
suburb = "Abbotsford-vic-3067"
driver.get(page_link)
current_url = driver.current_url 
for page in range(34,51):
    print("Going through page", page)
    #looping through pages of listings with a suburb

    time = datetime.now()
    file = open("ErrorLog.txt", "a")
    file.write('\nRan at ')
    file.write(str(time))
    file2 = open("UrlLog.txt", "a")
   
    
    if page == 1:
        driver.get(current_url)
    else:
        try:
            driver.get(current_url + "?page=" + str(page))
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[1]/div/div[1]/div[1]/div/div/div/div/div[2]/div/a/div/img')))

        except:
            file.write("\nGoing through pages of listings within suburbs broken at page" + str(page))
            break
            raise

    reset_point = driver.current_url


    for listing_num in range(1,25):
        listing_address = listing_description = listing_price = listing_amenities = listing_property_type = listing_sold_date = listing_domain_insight = listing_schools = listing_url = 0
        print(listings.shape[0])
        
        #looping through listings in a page
        driver.get(reset_point)

        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[' + str(listing_num) + ']/div/div[2]/div/a/h2/span[1]')))
            element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[' + str(listing_num) + ']/div/div[2]/div/a/h2/span[1]')
            element.click()
        
    #looping through listings in a page
            
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//button[@data-testid="listing-details__description-button"]')))
            driver.find_element_by_xpath('//button[@data-testid="listing-details__description-button"]').send_keys(u'\ue007')

            entire_listing = driver.find_element_by_css_selector("div.listing-details__content-container > div > div").text
            #print(entire_listing)

            listing_sold_date = driver.find_element_by_css_selector("div.listing-details__summary-title-container > div > span.listing-details__listing-tag.is-sold.listing-details__summary-tag").text
            #print(listing_sold_date)

            listing_price = driver.find_element_by_css_selector("div.listing-details__address-left-column > div").text
            #print(listing_price)

            listing_address = driver.find_element_by_css_selector("div.listing-details__copy-text > div").text
            #print(listing_address)

            listing_amenities = driver.find_element_by_css_selector("div.listing-details__summary-title-container > div > div.listing-details__listing-summary-features.css-er59q5 > div").text
            #print(listing_amenities)

            listing_description = driver.find_element_by_css_selector("div.listing-details__description > div.noscript-expander-wrapper.css-aeox7o > div > div > div").text
            #print(listing_description)

            listing_property_type = driver.find_element_by_css_selector("div.listing-details__property-type-features > span").text
            #print(listing_property_type)

            listing_domain_insight = driver.find_element_by_css_selector("div.listing-details__content-container > div > div > div.css-6s4w0i > div > div > p").text
            #print(listing_domain_insight)

            driver.find_element_by_xpath('//button[@data-testid="fe-co-school-catchment-view-more-link"]').send_keys(u'\ue007')
            listing_schools = driver.find_element_by_css_selector("div.listing-details__school-catchment > div > div > section").text
            #print(listing_schools)

            building_profile = driver.find_element_by_css_selector("#__domain_group\/APP_ROOT > div > div > div:nth-child(14)").text
            #print(building_profile)

            performance = driver.find_element_by_css_selector("div.listing-details__suburb-insights").text
            #print(performance)

            listing_url = driver.current_url

            data = [{'Suburb': suburb,'Address': listing_address, 
                     'Description': listing_description, 'Price': listing_price, 
                     'Amenities': listing_amenities, 'Type': listing_property_type, 
                     'Date': listing_sold_date, 'Insights': listing_domain_insight, 
                     'Schools': listing_schools, 'URL': listing_url, 'Profile': building_profile, 'Performance': performance}]
            
            output = pd.DataFrame(data) 
            listings = listings.append(output, ignore_index=True)

            file2.write("\n" + str(listing_url))
            file2.write("\n" + str(entire_listing) + "\n")
            
        except:
            file.write("\nError - " + str(listing_num) + " Not found in page" + str(page))
            pass

    listings.to_csv("databackup.csv", encoding='utf-8', index=False)
    file.close()
    file2.close()

Going through page 34
623
624
625
625
626
627
628
629
630
631
631
632
633
634
634
635
636
637
638
639
640
641
642
643
Going through page 35
643
644
645
645
646
647
648
649
650
651
651
652
653
654
654
655
656
657
658
659
660
661
662
663
Going through page 36
663
664
665
665
666
667
668
669
670
671
671
672
673
674
674
675
676
677
678
679
680
681
682
683
Going through page 37
683
684
685
685
686
687
688
689
690
691
691
692
693
694
694
695
696
697


In [38]:
file.close()
file2.close()

In [141]:
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//button[@data-testid="listing-details__description-button"]')))
driver.find_element_by_xpath('//button[@data-testid="listing-details__description-button"]').send_keys(u'\ue007')

entire_listing = driver.find_element_by_css_selector("div.listing-details__content-container > div > div").text
#print(entire_listing)

listing_sold_date = driver.find_element_by_css_selector("div.listing-details__summary-title-container > div > span.listing-details__listing-tag.is-sold.listing-details__summary-tag").text
#print(listing_sold_date)

listing_price = driver.find_element_by_css_selector("div.listing-details__address-left-column > div").text
#print(listing_price)

listing_address = driver.find_element_by_css_selector("div.listing-details__copy-text > div").text
print(listing_address)

listing_amenities = driver.find_element_by_css_selector("div.listing-details__summary-title-container > div > div.listing-details__listing-summary-features.css-er59q5 > div").text
#print(listing_amenities)

listing_description = driver.find_element_by_css_selector("div.listing-details__description > div.noscript-expander-wrapper.css-aeox7o > div > div > div").text
#print(listing_description)

listing_property_type = driver.find_element_by_css_selector("div.listing-details__property-type-features > span").text
#print(listing_property_type)

listing_domain_insight = driver.find_element_by_css_selector("div.listing-details__content-container > div > div > div.css-6s4w0i > div > div > p").text
#print(listing_domain_insight)

#driver.find_element_by_xpath('//button[@data-testid="fe-co-school-catchment-view-more-link"]').send_keys(u'\ue007')
listing_schools = driver.find_element_by_css_selector("div.listing-details__school-catchment > div > div > section").text
#print(listing_schools)

building_profile = driver.find_element_by_css_selector("#__domain_group\/APP_ROOT > div > div > div:nth-child(14)").text
#print(building_profile)

performance = driver.find_element_by_css_selector("div.listing-details__suburb-insights").text
#print(performance)

listing_url = driver.current_url

data = [{'Suburb': suburb,'Address': listing_address, 
         'Description': listing_description, 'Price': listing_price, 
         'Amenities': listing_amenities, 'Type': listing_property_type, 
         'Date': listing_sold_date, 'Insights': listing_domain_insight, 
         'Schools': listing_schools, 'URL': listing_url, 'Profile': building_profile, 'Performance': performance}]

listings = listings.append(output, ignore_index=True)


listings.to_csv("databackup.csv", encoding='utf-8', index=False)

627/20 Shamrock Street Abbotsford VIC 3067


In [ ]:
### Scrap Code Backup

for num_page in range(1,26):
    listing_address = listing_description = listing_price = listing_amenities = listing_property_type = listing_sold_date = listing_domain_insight = listing_schools = listing_url = 0
    
    #looping through listings in a page
    reset_point = driver.current_url
    
    try:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[' + str(num_page) + ']/div/div[2]/div/a/h2/span[1]')))
        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[' + str(num_page) + ']/div/div[2]/div/a/h2/span[1]')
        element.click()

        #Grabs listing description
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//button[@data-testid="listing-details__description-button"]')))
            driver.find_element_by_xpath('//button[@data-testid="listing-details__description-button"]').send_keys(u'\ue007')
            WebDriverWait(driver, 1)
        except:
            pass
        listing_description = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[5]/div/div/div[3]/div[1]/div/div/div').text

        #Grabs listing's address
        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[2]/div/div[1]')
        listing_address = element.text

        #Grabs listing's price
        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[1]/div[1]/div')
        listing_price = element.text

        #Grabs listing's Amenities
        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div')
        listing_amenities = element.text

        #Grabs listing's property type
        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[4]/span')
        listing_property_type = element.text

        #Grabs listing's sold date
        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/span[2]')
        listing_sold_date = element.text
        
        try:
            #Grabs listing's domain insight
            element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[5]/div/div/p')
            listing_domain_insight = element.text

            #Grabs School details
            driver.find_element_by_xpath('//button[@data-testid="fe-co-school-catchment-view-more-link"]').send_keys(u'\ue007')
            element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[9]/div/div/section/div[3]/ul')
            listing_schools = element.text
        except:
            pass

        #Grabs listing url
        listing_url = driver.current_url

        #Storing data
        data = [{'Suburb': suburb,'Address': listing_address, 'Description': listing_description, 'Price': listing_price, 'Amenities': listing_amenities, 'Type': listing_property_type, 'Date': listing_sold_date, 'Insights': listing_domain_insight, 'Schools': listing_schools, 'URL': listing_url}] 
        output = pd.DataFrame(data)    
        listings = listings.append(output, ignore_index=True)
        
    except:
        pass

    driver.get(reset_point)

In [36]:
listings.shape

(623, 12)

In [37]:
listings.to_csv("databackup.csv", encoding='utf-8', index=False)

In [27]:
latest_run == listings

,Suburb,Address,Description,Price,Amenities,Type,Date,Insights,Schools,URL,Profile,Performance
0,False,False,False,False,False,False,False,False,False,False,False,False
1,True,False,True,True,True,True,True,True,True,True,True,True
2,True,False,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
599,True,True,True,True,True,True,True,True,True,True,True,True
600,True,True,True,True,True,True,True,True,True,True,False,True
601,True,True,True,True,True,True,True,True,True,True,False,True
602,True,True,True,True,True,True,True,True,True,True,True,True


In [32]:
listings.to_json("data.json")

In [33]:
pd.read_csv("data.json")

,"{""Suburb"":{""0"":0","1:""Abbotsford-vic-3067""","2:""Abbotsford-vic-3067""","3:""Abbotsford-vic-3067""","4:""Abbotsford-vic-3067""","5:""Abbotsford-vic-3067""","6:""Abbotsford-vic-3067""","7:""Abbotsford-vic-3067""","8:""Abbotsford-vic-3067""","9:""Abbotsford-vic-3067""",...,"599:""Insights for Abbotsford\nMarket performance\nMEDIAN PRICE\n: \n-\nAUCTION CLEARANCE\n: \n-\nSOLD THIS YEAR\n: \n-\nAVG DAYS ON MARKET\n: \n-\n*Based on 1 bedroom s in Abbotsford\nDemographics\nPOPULATION\n: \n8","173\nOccupancy: \nOWNER\n: \n40%\nRENTER\n: \n60%\nAVERAGE AGE\n: \n20 to 39\nHousehold: \nFAMILY\n: \n26%\nSINGLE\n: \n74%\n*Based on general data in Abbotsford\nView suburb profile"".599","600:""Insights for 2 bedroom units in Abbotsford\nMarket performance\nMEDIAN PRICE\n: \n$560k\nAUCTION CLEARANCE\n: \n67%\nSOLD THIS YEAR\n: \n94\nAVG DAYS ON MARKET\n: \n176\n*Based on 2 bedroom Units in Abbotsford\nDemographics\nPOPULATION\n: \n8","173\nOccupancy: \nOWNER\n: \n40%\nRENTER\n: \n60%\nAVERAGE AGE\n: \n20 to 39\nHousehold: \nFAMILY\n: \n26%\nSINGLE\n: \n74%\n*Based on general data in Abbotsford\nView suburb profile"".600","601:""Insights for 2 bedroom houses in Abbotsford\nMarket performance\nMEDIAN PRICE\n: \n$1.02m\nAUCTION CLEARANCE\n: \n74%\nSOLD THIS YEAR\n: \n32\nAVG DAYS ON MARKET\n: \n14\n*Based on 2 bedroom Houses in Abbotsford\nDemographics\nPOPULATION\n: \n8","173\nOccupancy: \nOWNER\n: \n40%\nRENTER\n: \n60%\nAVERAGE AGE\n: \n20 to 39\nHousehold: \nFAMILY\n: \n26%\nSINGLE\n: \n74%\n*Based on general data in Abbotsford\nView suburb profile"".601","602:""Insights for 2 bedroom units in Abbotsford\nMarket performance\nMEDIAN PRICE\n: \n$560k\nAUCTION CLEARANCE\n: \n67%\nSOLD THIS YEAR\n: \n94\nAVG DAYS ON MARKET\n: \n176\n*Based on 2 bedroom Units in Abbotsford\nDemographics\nPOPULATION\n: \n8","173\nOccupancy: \nOWNER\n: \n40%\nRENTER\n: \n60%\nAVERAGE AGE\n: \n20 to 39\nHousehold: \nFAMILY\n: \n26%\nSINGLE\n: \n74%\n*Based on general data in Abbotsford\nView suburb profile"".602","603:""Insights for 2 bedroom units in Abbotsford\nMarket performance\nMEDIAN PRICE\n: \n$560k\nAUCTION CLEARANCE\n: \n67%\nSOLD THIS YEAR\n: \n94\nAVG DAYS ON MARKET\n: \n176\n*Based on 2 bedroom Units in Abbotsford\nDemographics\nPOPULATION\n: \n8","173\nOccupancy: \nOWNER\n: \n40%\nRENTER\n: \n60%\nAVERAGE AGE\n: \n20 to 39\nHousehold: \nFAMILY\n: \n26%\nSINGLE\n: \n74%\n*Based on general data in Abbotsford\nView suburb profile""}}"
